In [1]:
import urllib.request as request
import json
import boto3
import pandas as pd

In [2]:
s3_stage = boto3.client('s3')
#then list all buckets
bucket_response = s3_stage.list_buckets()
buckets = bucket_response["Buckets"]   

In [3]:
#then find the bucket containing 'data-stage-bucket'
correct_bucket = []
for i in range(0,len(buckets)):
    if 'data-dump-bucket' in buckets[i]['Name']:
        correct_bucket.append(buckets[i]['Name'])
    else:
        continue
    bucket_name_dump = correct_bucket[0]  

In [4]:
bucket_name_dump

'capstone-team-uk-demo-7-data-dump-bucket'

In [5]:
s3_read = boto3.resource('s3')
itemname = 'raw-zone/timestamp_file_covid.txt'
obj = s3_read.Object(bucket_name_dump, itemname)
body_timestamp = obj.get()['Body'].read()
timestamp = body_timestamp.decode()

In [6]:
timestamp

'2020-08-05_07_25_PM'

In [10]:
obj

s3.Object(bucket_name='capstone-team-uk-demo-5-data-dump-bucket', key='raw-zone/timestamp_file_covid.txt')

In [11]:
covid_filename = 'covid-data-'+timestamp+'.json'
itemname = 'raw-zone/'+covid_filename
obj = s3_read.Object(bucket_name_dump, itemname)
body = obj.get()['Body'].read()

In [12]:
body

b'{"objectIdFieldName": "OBJECTID", "uniqueIdField": {"name": "OBJECTID", "isSystemMaintained": true}, "globalIdFieldName": "", "geometryProperties": {"shapeAreaFieldName": "Shape__Area", "shapeLengthFieldName": "Shape__Length", "units": "esriDecimalDegrees"}, "geometryType": "esriGeometryPolygon", "spatialReference": {"wkid": 4326, "latestWkid": 4326}, "fields": [{"name": "OBJECTID", "type": "esriFieldTypeOID", "alias": "OBJECTID", "sqlType": "sqlTypeOther", "domain": null, "defaultValue": null}, {"name": "FULL_", "type": "esriFieldTypeString", "alias": "FULL", "sqlType": "sqlTypeOther", "length": 50, "domain": null, "defaultValue": null}, {"name": "GEOID", "type": "esriFieldTypeString", "alias": "GEOID", "sqlType": "sqlTypeOther", "length": 5, "domain": null, "defaultValue": null}, {"name": "LABEL", "type": "esriFieldTypeString", "alias": "LABEL", "sqlType": "sqlTypeOther", "length": 35, "domain": null, "defaultValue": null}, {"name": "STAETFP", "type": "esriFieldTypeString", "alias"

In [13]:
data = json.loads(body)
json_simple = data['features']
new_list_of_dicts = [x['attributes'] for x in json_simple]

In [15]:
data_f = pd.DataFrame(new_list_of_dicts)
data_f.index.name = 'id'

In [18]:
#filter out non-county values
filter_list = ['OUT OF STATE', 'UNKNOWN', 'INTERNATIONAL']
#two ways to filter data frames
data_f_filter = data_f[~data_f['LABEL'].isin(filter_list)]
data_f_filter_chain = data_f[~data_f.LABEL.isin(filter_list)]

In [22]:
test = data_f_filter['OBJECTID']

In [27]:
data_f_filter.to_csv('covid-data.csv',index=False)

In [7]:
#kms named keys to assign in lambda
kms_client = boto3.client('kms')

In [18]:
response = kms_client.list_keys()['Keys']
print(response)

[{'KeyId': '097a4600-366a-4924-a1e3-0f531ce5926e', 'KeyArn': 'arn:aws:kms:us-west-2:831466106657:key/097a4600-366a-4924-a1e3-0f531ce5926e'}, {'KeyId': 'b28006a8-52bb-4a73-b1f2-4146b06c2b93', 'KeyArn': 'arn:aws:kms:us-west-2:831466106657:key/b28006a8-52bb-4a73-b1f2-4146b06c2b93'}, {'KeyId': 'd99bbcad-c60d-416b-9a7f-8413e2802a5d', 'KeyArn': 'arn:aws:kms:us-west-2:831466106657:key/d99bbcad-c60d-416b-9a7f-8413e2802a5d'}]


In [20]:
keys_list = []
for i in range(0,len(response)):
    keys_list.append(response[i]['KeyId'])
    
print(keys_list)

['097a4600-366a-4924-a1e3-0f531ce5926e', 'b28006a8-52bb-4a73-b1f2-4146b06c2b93', 'd99bbcad-c60d-416b-9a7f-8413e2802a5d']


In [45]:
response_tag_list = []
tag_dict = {}
for i in range(0, len(keys_list)):
    response_tag_list.append(kms_client.list_resource_tags(KeyId = keys_list[i])['Tags'])
    tag_dict[keys_list[i]] = response_tag_list[i]
print(tag_dict)

{'097a4600-366a-4924-a1e3-0f531ce5926e': [], 'b28006a8-52bb-4a73-b1f2-4146b06c2b93': [{'TagKey': 'Name', 'TagValue': 'capstone_key'}], 'd99bbcad-c60d-416b-9a7f-8413e2802a5d': []}


In [43]:
kms_client.list_resource_tags(KeyId = keys_list[0])['Tags']

[]

In [36]:
tag_dict

{'097a4600-366a-4924-a1e3-0f531ce5926e': [],
 'b28006a8-52bb-4a73-b1f2-4146b06c2b93': [{'TagKey': 'Name',
   'TagValue': 'capstone_key'}],
 'd99bbcad-c60d-416b-9a7f-8413e2802a5d': []}

In [18]:
keyid1 = response['Keys'][0]['KeyId']
keyid2 = response['Keys'][1]['KeyId']

In [22]:
response_tags = kms_client.list_resource_tags(KeyId = keyid2)

In [23]:
response_tags['Tags'][0]['TagValue']

'capstone_key'

In [25]:
response

{'Keys': [{'KeyId': '54e698dd-00f8-443e-8109-6aaab892ae38',
   'KeyArn': 'arn:aws:kms:us-west-2:064865197051:key/54e698dd-00f8-443e-8109-6aaab892ae38'},
  {'KeyId': 'd89f6a63-2390-4e9a-8bb1-13504f50785c',
   'KeyArn': 'arn:aws:kms:us-west-2:064865197051:key/d89f6a63-2390-4e9a-8bb1-13504f50785c'}],
 'Truncated': False,
 'ResponseMetadata': {'RequestId': '0c25fd15-87e4-413d-8689-a8df7584f35b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '0c25fd15-87e4-413d-8689-a8df7584f35b',
   'cache-control': 'no-cache, no-store, must-revalidate, private',
   'expires': '0',
   'pragma': 'no-cache',
   'date': 'Wed, 5 Aug 2020 14:46:07 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '313'},
  'RetryAttempts': 0}}